# 5. Bidding Strategy tutorial

In this part of the tutorial, we formulate a rule-based bidding strategy for the Electrolyser unit. Rule-based strategies guide the unit's market participation by following a predefined set of rules and guidelines, enabling it to make market offers autonomously.


A rule-based bidding strategy is a set of algorithmic guidelines that dictate how a unit should participate in the electricity market. This is particularly important for:

- **Automated Decision-making**: Allows the unit to make bids autonomously, reducing the need for human intervention.
- **Market Adaptability**: Enables the unit to adjust its bidding behavior in response to dynamic market conditions.

**Key Components to consider to establish Rule-Based Bidding Strategy**

1. **Product Type**: This sets the stage for the kind of products (e.g., energy, ancillary services) that the unit will bid for in the electricity market.
2. **Bid Volume**: The quantity of the product that the unit offers in its bid.
3. **Marginal Revenue**: Used to determine the price at which the unit should make its bid. This involves calculating the unit's incremental revenue for additional units of electricity sold or consumed.


In [ ]:
#Some important imports
from assume.common.base import BaseStrategy, SupportsMinMax
from assume.common.market_objects import MarketConfig, Order, Orderbook, Product

In [ ]:
# install JDC that allows us defining functions of classes across different cells
%pip install jdc

## 1. Define `NaiveStrategyElectrolyser` Class
The `NaiveStrategyElectrolyser` class inherits from the `BaseStrategy` class and implements the `calculate_bids` method, which is responsible for formulating the market bids.

### 1.1. Key Methods and Parameters
- `calculate_bids`: This method takes several arguments, including the unit to be dispatched (`unit`), the market configuration (`market_config`), and a list of products (`product_tuples`). It returns an `Orderbook` containing the bids.


In [ ]:
class NaiveStrategyElectrolyser(BaseStrategy):
    """
    A naive strategy that bids the marginal cost of the electrolyser on the market.
    """

    def calculate_bids(
        self,
        unit: SupportsMinMax,
        market_config: MarketConfig,
        product_tuples: list[Product],
        **kwargs,
    ) -> Orderbook:
        
        """
        Takes information from a unit that the unit operator manages and
        defines how it is dispatched to the market
        :param unit: the unit to be dispatched
        :type unit: SupportsMinMax
        :param market_config: the market configuration
        :type market_config: MarketConfig
        :param product_tuples: list of all products the unit can offer
        :type product_tuples: list[Product]
        :return: the bids
        :rtype: Orderbook
        """

In [ ]:
start = product_tuples[0][0]  # start time of the first product

### 1.1.1 Calculating Marginal Revenue: The Mathematical Equation

Calculating marginal revenue is crucial for grasping the economics of the bidding strategy. The formula used in the code is as follows:

\[
\text{Marginal Revenue} = \left( \text{Hydrogen Price} - \text{Fixed Cost} \right) \times \frac{\text{Hydrogen Production}}{\text{Power}}
\]

**Parameters Explained** 

- **Hydrogen Price**: The price of hydrogen at the specific time frame, fetched from the unit's forecaster.
- **Fixed Cost**: The constant cost associated with the unit, not varying with the amount of power or hydrogen produced.
- **Hydrogen Production**: The amount of hydrogen produced during the given time frame, calculated based on the hydrogen demand.
- **Power**: The electrical power consumed by the Electrolyser unit to produce the given amount of hydrogen.

**Why is This Formula Important?**

1. **Economic Efficiency**: This formula helps in determining the most economically efficient way to bid in the electricity market.
2. **Optimal Pricing**: Knowing your marginal revenue enables you to set your bid price optimally, maximizing profitability while maintaining competitiveness.
3. **Demand-Side Management (DSM)**: Accurate calculation of marginal revenue is essential for effective DSM strategies, allowing the unit to respond appropriately to market signals.


In [ ]:
bids = []
for product in product_tuples:
    """
    for each product, calculate the marginal revenue of the unit at the start time of the product
    and the volume of the product. Dispatch the order to the market.
    """

**1. Fetch Start and End Times:** For each product, the loop fetches the start and end times.

In [ ]:
start = product[0]
end = product[1]

**2. Retrieve Hydrogen Demand and Price:** The hydrogen demand and price are fetched from the unit's forecaster for the corresponding start time.

In [ ]:
# Calculate marginal cost using the adjusted efficiency
hydrogen_demand = unit.forecaster[f"{unit.id}_h2demand"].loc[start]
hydrogen_price = unit.forecaster[f"{unit.id}_h2price"].loc[start]

**3. Calculate Bid Volume:** The code calls unit.calculate_min_max_power to determine the bid volume based on the hydrogen demand.

In [ ]:
power, hydrogen_production = unit.calculate_min_max_power(
    start=start,
    end=end,
    hydrogen_demand=hydrogen_demand,
)

### 1.1.2 Calculate Marginal Revenue and Bid Price: Using the formula discussed in the previous section, the marginal revenue is calculated to determine the bid price

In [ ]:
marginal_revenue = (hydrogen_price - unit.fixed_cost) * hydrogen_production / power

In [ ]:
bid_price = marginal_revenue

### 1.1.3 Creating Market Orders and Finalizing Bids

In this segment of the tutorial, we shall explore how market orders are generated and added to the bids list. This step is crucial as it translates our calculated bid volume and price into actionable market orders.

The `Order` dictionary object encapsulates all the necessary information for a single bid. It includes the following key-value pairs:

- **`start_time` and `end_time`**: The time window for which the bid is valid.
- **`volume`**: The bid volume, which is the power calculated based on hydrogen demand.
- **`price`**: The bid price, determined by the marginal revenue.

In [ ]:
order: Order = {
    "start_time": product[0],
    "end_time": product[1],
    "only_hours": product[2],
    "price": bid_price,
    "volume": -power,
}
bids.append(order)